<h1 style="text-align: center; color: RED">ENCODİNG</h1>

Encode etmek değişkenlerin temsil şekillerinin değiştirilmesi demektir.

<span style="color: blue; font-weight: bold;">1-LABEL ENCODİNG</span>

Label, etiket demektir. Yani kategorik değişkenlerdeki sınıflara label(etiket) deriz.Label Encoding, kategorik (metinsel) değişkenlerin, makine öğrenmesi algoritmaları tarafından kullanılabilmesi için sayısal değerlere dönüştürülmesidir. Makine öğrenmesi algoritmaları metinsel verilerle çalışamaz, bu yüzden sayıya çevrilmeleri gerekir.
![Test](datasets/6.png)

Ordinal olan(sıralı, sınıflar arasında fark olan) kategorik değişkenlerin encode edilmesi:
![Test](datasets/7.png)

Aşağıdaki tabloda verilen değişkenlerin label encodingten geçirilmesi yanlış olacaktır. Çünkü takımlar sıralı yani ordinal değişken değildir. Bu sebeple label encoding yapılması dogru değildir.
![Test](datasets/8.png)

<span style="color: blue; font-weight: bold;">LABEL ENCODİNG & BINARY ENCODING UYGULAMA</span>

Eğer kategorik değişkende iki sınıf varsa buna bınary encodingde diyebiliriz. 2 den fazla sınıf varsa bu da label encodingdir.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno #eksik değerler için kullanılan bir kütüphane
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

#Görsel ayaralama için
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
def load_application_train():
    data = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\6.Feature Engineering\\datasets\\application_train.csv")
    return data

In [5]:
def load():
    data = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\6.Feature Engineering\\datasets\\titanic.csv")
    return data

In [7]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [26]:
#Bir değişkene label encoding uygulayalım
df['Sex'].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [28]:
le = LabelEncoder()

In [30]:
le.fit_transform(df['Sex'])[0:5]

array([1, 0, 0, 0, 1])

* Encode işlemi yaparken alfabetik sıraya göre ilk gördüğü değere 0 diğere değere 1 verir.

In [33]:
le.inverse_transform([0,1])# dönüştürdükleri değerin karşılığını öğrenmek için

array(['female', 'male'], dtype=object)

In [9]:
#Fonksiyonlaştırma
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[col] = labelencoder.fit_transform(dataframe[col])
    return dataframe
    

In [36]:
def grab_col_names(dataframe, cat_th=10, car_th=20):#parametrelerin ön tanımlı değerlerini burda belirttik

    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['object', 'category', 'bool']]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].dtypes in ['int64', 'float64'] and dataframe[col].nunique() < cat_th]
    cat_but_car = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['object', 'category'] and dataframe[col].nunique() > car_th]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car] 

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes  in ['int64', 'float64']] #bu sayısal değişkenleri verir
    num_cols = [col for col in num_cols if col not in cat_cols] 

    print(f'Observetions: {dataframe.shape[0]}')
    print(f'Variables: {dataframe.shape[1]}')
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car


In [43]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observetions: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
num_but_cat: 4


In [45]:
cat_cols

['Sex', 'Embarked', 'Survived', 'Pclass', 'SibSp', 'Parch']

In [63]:
binary_col = [col for col in cat_cols if df[col].nunique() ==2 and df[col].dtype=='O'] # 2 sınıfı olan ve sayısal olarak ifade edilmeyen kategorik değişkenleri seçtik
    

In [61]:
binary_col

['Sex']

In [13]:
#Başka bir veri seti üzerinde inceliyoruz
dff = load_application_train()

In [15]:
cat_cols, num_cols, cat_but_car = grab_col_names(dff)

Observetions: 307511
Variables: 122
cat_cols: 54
num_cols: 67
cat_but_car: 1
num_but_cat: 39


In [17]:
binary_col = [col for col in cat_cols if dff[col].nunique() ==2 and dff[col].dtype=='O'] # 2 sınıfı olan ve sayısal olarak ifade edilmeyen kategorik değişkenleri seçtik
    

In [75]:
binary_col

['NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'EMERGENCYSTATE_MODE']

In [ ]:
dff[binary_col]

In [77]:
for col in binary_col:
    label_encoder(dff, col)

In [ ]:
dff.head()

<span style="color: blue; font-weight: bold;">2-ONEHOT ENCODİNG</span>

One-Hot Encoding, makine öğrenmesi modellerinde kategorik (sınıflandırılmış) verilerin sayısal olarak ifade edilmesini sağlayan etkili bir yöntemdir. Bu yöntem, bir kategorik değişkendeki her bir benzersiz değeri (etiketi) ayrı bir sütuna dönüştürür ve bu sütunlarda sadece 0 ve 1 değerleri yer alır. Bir gözlem, ait olduğu kategoriye karşılık gelen sütunda “1”, diğer tüm sütunlarda ise “0” olarak temsil edilir. Örneğin, “renk” adında ve içinde `kırmızı`, `mavi`, `yeşil` gibi değerler olan bir değişken düşünelim. One-Hot Encoding uygulandığında, bu üç renk için üç ayrı sütun oluşur ve her gözlemde yalnızca ait olduğu rengin sütununda 1 olur. Böylece model, kategorilerin birbirine üstünlük ya da büyüklük ilişkisi olmadığını doğru bir şekilde algılar.

Dummy tuzağı (dummy variable trap), One-Hot Encoding gibi yöntemlerle oluşturulan kategorik değişkenlerin yanlış şekilde kullanılması sonucu oluşan çoklu doğrusal bağlantı (multicollinearity) problemidir. Bu, bazı değişkenlerin birbirine tamamen bağımlı hâle gelmesiyle oluşur ve özellikle doğrusal modellerde (Linear Regression, Logistic Regression gibi) modelin yanlış öğrenmesine neden olabilir.One-Hot Encoding sonrası, her kategorinin bir sütunu yerine (k - 1) adet sütun bırakılır. Yani örneğimizde 3 kategori varsa, 2 sütun kalır, biri referans (baseline) olarak alınır.
![Test](datasets/9.png)

<span style="color: blue; font-weight: bold;">2-ONEHOT ENCODİNG UYGULAMA</span>

In [16]:
df = load()

In [20]:
df['Embarked'].value_counts()#3 tane sınıf var ve sınıflar arasında fark yok yani nominal sınıflar

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

* Yukarıdaki kategorik değişken label encoding yöntemiyle dönüştürüldüğünde, sınıflara 1, 2, 3 gibi ardışık sayısal değerler atanır. Ancak bu dönüşüm, aslında kendi aralarında herhangi bir sıralı veya nicel ilişki bulunmayan sınıflar arasında yapay bir büyüklük veya hiyerarşi algısı yaratır. Bu durum da modelin yanlış öğrenmesine neden olabilir.

In [24]:
pd.get_dummies(df, columns = ['Embarked'], drop_first=True).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,False,True
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,False,True


* drop_first=True parametresi, dummy değişken tuzağını (dummy variable trap) önlemek için kullanılır. Bu parametre:

Oluşturulan ilk kategorik sütunu otomatik olarak siler.

Yani toplam n kategori varsa, sadece n-1 tane dummy sütun bırakır.

Bu, modellerde çoklu doğrusal bağlantı (multicollinearity) oluşmasını önler.



In [ ]:
pd.get_dummies(df, columns = ['Embarked'], dummy_na=True).head()#eksik değerlerde bir sınıf olarak gelsin istersek

* Not : binary olanları da one hot encoding ile yapabiliriz.drop_first=True parametresini kullanmamız yeterli olacaktır.

In [28]:
#Fonksiyonlaştırma

def one_hot_encoder(dataframe, cat_col, drop_first=True):
    dataframe = pd.get_dummies(dataframe,columns = cat_col, drop_first = drop_first)
    return dataframe

In [30]:
df = load()

In [38]:
cat_cols, num_col,cat_but_car = grab_col_names(df)

Observetions: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
num_but_cat: 4


In [40]:
cat_cols

['Sex', 'Embarked', 'Survived', 'Pclass', 'SibSp', 'Parch']

In [42]:
ohe_cols = [col for col in df.columns if 10>= df[col].nunique()>2]

In [44]:
ohe_cols

['Pclass', 'SibSp', 'Parch', 'Embarked']

In [46]:
one_hot_encoder(df, ohe_cols)

,PassengerId,Survived,Name,Sex,Age,Ticket,Fare,Cabin,Pclass_2,Pclass_3,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",male,22.000,A/5 21171,7.250,NaN,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,PC 17599,71.283,C85,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,1,"Heikkinen, Miss. Laina",female,26.000,STON/O2. 3101282,7.925,NaN,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,113803,53.100,C123,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
4,5,0,"Allen, Mr. William Henry",male,35.000,373450,8.050,NaN,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True
5,6,0,"Moran, Mr. James",male,NaN,330877,8.458,NaN,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False
6,7,0,"McCarthy, Mr. Timothy J",male,54.000,17463,51.862,E46,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
7,8,0,"Palsson, Master. Gosta Leonard",male,2.000,349909,21.075,NaN,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,True
8,9,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000,347742,11.133,NaN,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True
9,10,1,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000,237736,30.071,NaN,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


<span style="color: blue; font-weight: bold;">3-RARE ENCODİNG</span>

Rare encoding (veya rare category encoding) kategorik değişkenlerde çok nadir görülen kategorileri tespit edip bunları tek bir ortak kategori altında toplama işlemidir. Amaç, veri setindeki seyrek (rare) kategorilerin modele olumsuz etkisini azaltmaktır.Kategorik değişken çok fazla kategoriye sahipse ve bunların bazıları çok nadir ise rare encoding tercih edilir.

Neden Yapılır?

Kategorik değişkenlerde bazı sınıflar çok az sayıda gözleme sahip olabilir. Bu durum:

Aşırı öğrenme (overfitting) riskini artırır → Model, nadir kategoriye ait birkaç örneği ezberler.

Genelleme yeteneğini düşürür → Test verisinde o kategori hiç çıkmayabilir, model zorlanır.

İstatistiksel anlamlılığı azaltır → Regresyon gibi modellerde katsayı tahminleri güvenilmez hale gelir.

Veri dengesizliğine yol açar → Özellikle sınıflandırma problemlerinde hatalı tahminler artabilir.

Nasıl Yapılır?

Eşik belirlenir (ör. %1’den az görünen kategoriler nadir sayılabilir).

Bu nadir kategoriler tek bir isim altında toplanır, genellikle "Rare" veya "Other" olarak adlandırılır.
![Test](datasets/10.png)

<span style="color: blue; font-weight: bold;">3-RARE ENCODİNG UYGULAMA</span>

Bu bölümü üç adımda gercekleştiricez:

 1. Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.
 2. Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
 3. Rare encoder yazacağız.


In [56]:
#1. adım Kategorik değişkenlerin azlık çokluk durumunun analiz edilmesi.

df = load_application_train()

In [58]:
df['NAME_EDUCATION_TYPE'].value_counts() #sınıflar ve frkanslarını görnek için yaptım

NAME_EDUCATION_TYPE
Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: count, dtype: int64

In [60]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observetions: 307511
Variables: 122
cat_cols: 54
num_cols: 67
cat_but_car: 1
num_but_cat: 39


In [62]:
cat_cols#kategorik değişkneleri yakaladım

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'TARGET',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOC

In [64]:
#Kategorik değişkenler ve sınıf oranları
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

In [66]:
for col in cat_cols:
    cat_summary(df, col)

                    NAME_CONTRACT_TYPE  Ratio
NAME_CONTRACT_TYPE                           
Cash loans                      278232 90.479
Revolving loans                  29279  9.521
##########################################
             CODE_GENDER  Ratio
CODE_GENDER                    
F                 202448 65.834
M                 105059 34.164
XNA                    4  0.001
##########################################
              FLAG_OWN_CAR  Ratio
FLAG_OWN_CAR                     
N                   202924 65.989
Y                   104587 34.011
##########################################
                 FLAG_OWN_REALTY  Ratio
FLAG_OWN_REALTY                        
Y                         213312 69.367
N                          94199 30.633
##########################################
                 NAME_TYPE_SUITE  Ratio
NAME_TYPE_SUITE                        
Unaccompanied             248526 80.819
Family                     40149 13.056
Spouse, partner            1

* Çıktıyı gözlemlediğimizde bazı sınıfların oranları 0.001 yani çok düşük. Bunları one hot encoder dan gecirirsek gerekli gereksiz bir sürü değişken oluşur. Buda bize ölçüm problemi çıkarrır.Bu sayısı az olan sınıfların bizim için ayırt edici özellik taşımıyor olabilir.

In [71]:
# Rare kategoriler ile bağımlı değişken arasındaki ilişkinin analiz edilmesi.
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

In [73]:
rare_analyser(df, "TARGET", cat_cols)

NAME_CONTRACT_TYPE : 2
                     COUNT  RATIO  TARGET_MEAN
NAME_CONTRACT_TYPE                            
Cash loans          278232  0.905        0.083
Revolving loans      29279  0.095        0.055


CODE_GENDER : 3
              COUNT  RATIO  TARGET_MEAN
CODE_GENDER                            
F            202448  0.658        0.070
M            105059  0.342        0.101
XNA               4  0.000        0.000


FLAG_OWN_CAR : 2
               COUNT  RATIO  TARGET_MEAN
FLAG_OWN_CAR                            
N             202924  0.660        0.085
Y             104587  0.340        0.072


FLAG_OWN_REALTY : 2
                  COUNT  RATIO  TARGET_MEAN
FLAG_OWN_REALTY                            
N                 94199  0.306        0.083
Y                213312  0.694        0.080


NAME_TYPE_SUITE : 7
                  COUNT  RATIO  TARGET_MEAN
NAME_TYPE_SUITE                            
Children           3267  0.011        0.074
Family            40149  0.131      

In [77]:
# 3. Adım Rare encoder yapma
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

In [79]:
new_df = rare_encoder(df, 0.01)

In [81]:
rare_analyser(new_df, "TARGET", cat_cols)

NAME_CONTRACT_TYPE : 2
                     COUNT  RATIO  TARGET_MEAN
NAME_CONTRACT_TYPE                            
Cash loans          278232  0.905        0.083
Revolving loans      29279  0.095        0.055


CODE_GENDER : 3
              COUNT  RATIO  TARGET_MEAN
CODE_GENDER                            
F            202448  0.658        0.070
M            105059  0.342        0.101
Rare              4  0.000        0.000


FLAG_OWN_CAR : 2
               COUNT  RATIO  TARGET_MEAN
FLAG_OWN_CAR                            
N             202924  0.660        0.085
Y             104587  0.340        0.072


FLAG_OWN_REALTY : 2
                  COUNT  RATIO  TARGET_MEAN
FLAG_OWN_REALTY                            
N                 94199  0.306        0.083
Y                213312  0.694        0.080


NAME_TYPE_SUITE : 5
                  COUNT  RATIO  TARGET_MEAN
NAME_TYPE_SUITE                            
Children           3267  0.011        0.074
Family            40149  0.131      

<h1 style="text-align: center; color: RED">ÖZELLİK ÖLÇEKLENDİRME(FEATURE SCALİNG)</h1>

Özellik ölçeklendirmedeki amaçlardan birisi değişkenler arasındaki ölçüm farklılığını gidermektir. Kullanılacak modellerin değişkenlere eşit şartlarda yaklaşmasını sağlamamız gerekmektedir.
![Test](datasets/11.png)

Diğer bir amacımız ise özellikle gradient descent kullanan algoritmaların train sürelerini kısaltmaktır.
![Test](datasets/12.png)

KNN gibi uzaklık temelli yöntemlerde scale yapmamak yanlılığa sebep olabilir. Bu sebeple verimizi ölçeklendirmeliyiz.

<span style="color: blue; font-weight: bold;">1-STANDART SCALER</span>

StandardScaler, makine öğrenmesinde kullanılan ölçeklendirme (scaling) yöntemlerinden biridir ve verileri ortalaması 0, standart sapması 1 olacak şekilde dönüştürür.

Standartlaştırma, şu formülle yapılır:

z= σ/x−μ

x → orijinal değer

μ → sütunun ortalaması

σ → sütunun standart sapması

Bu işlem sonucunda:

Ortalama (mean) = 0

Standart sapma (std) = 1

Değerler, normal dağılıma daha yakın hale gelir (özellikle veri zaten Gaussian dağılımına yakınsa).



In [90]:
df = load()
ss = StandardScaler()
df["Age_standard_scaler"] = ss.fit_transform(df[["Age"]])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_standard_scaler
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,-0.530
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,0.572
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,-0.255
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,0.365
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0.365


<span style="color: blue; font-weight: bold;">2-ROBUST SCALER</span>

RobustScaler, verileri ölçeklendirirken ortalama ve standart sapma yerine medyan ve IQR (Interquartile Range) kullanır.
Formülü şu şekildedir:

x−Medyan /IQR

 
Medyan → Ortanca değer

IQR (Interquartile Range) = Q3 − Q1 (3. çeyrek - 1. çeyrek)

| Özellik        | StandardScaler                                      | RobustScaler                                        |
| -------------- | --------------------------------------------------- | --------------------------------------------------- |
| Merkezleme     | Ortalama (mean)                                     | Medyan                                              |
| Ölçekleme      | Standart sapma (std)                                | IQR                                                 |
| Outlier etkisi | **Yüksek** (uç değerler ortalamayı ve std’yi bozar) | **Düşük** (medyan ve IQR uç değerlerden etkilenmez) |
| Kullanım alanı | Normal dağılıma yakın veriler                       | Uç değer (outlier) içeren veriler                   |

Aykırı değerlerden etkilenmemektedir.

In [95]:
rs = RobustScaler()
df["Age_robuts_scaler"] = rs.fit_transform(df[["Age"]])
df.describe().T


,count,mean,std,min,25%,50%,75%,max
PassengerId,891.000,446.000,257.354,1.000,223.500,446.000,668.500,891.000
Survived,891.000,0.384,0.487,0.000,0.000,0.000,1.000,1.000
Pclass,891.000,2.309,0.836,1.000,2.000,3.000,3.000,3.000
Age,714.000,29.699,14.526,0.420,20.125,28.000,38.000,80.000
SibSp,891.000,0.523,1.103,0.000,0.000,0.000,1.000,8.000
Parch,891.000,0.382,0.806,0.000,0.000,0.000,0.000,6.000
Fare,891.000,32.204,49.693,0.000,7.910,14.454,31.000,512.329
Age_standard_scaler,714.000,0.000,1.001,-2.017,-0.660,-0.117,0.572,3.465
Age_robuts_scaler,714.000,0.095,0.813,-1.543,-0.441,0.000,0.559,2.909


<span style="color: blue; font-weight: bold;">3-MİN-MAX SCALER</span>

MinMaxScaler, verileri belirli bir aralığa (genellikle 0 ile 1) ölçeklendiren bir yöntemdir.
Formülü şöyledir:


 x−xmin/ xmax - xmin

x → orijinal değer

x min→ sütundaki minimum değer

𝑥max→ sütundaki maksimum değer

Sonuç olarak:

Minimum değer 0 olur.

Maksimum değer 1 olur.

Diğer değerler bu aralıkta yeniden ölçeklenir.

In [101]:
mms = MinMaxScaler()
df["Age_min_max_scaler"] = mms.fit_transform(df[["Age"]])
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.000,446.000,257.354,1.000,223.500,446.000,668.500,891.000
Survived,891.000,0.384,0.487,0.000,0.000,0.000,1.000,1.000
Pclass,891.000,2.309,0.836,1.000,2.000,3.000,3.000,3.000
Age,714.000,29.699,14.526,0.420,20.125,28.000,38.000,80.000
SibSp,891.000,0.523,1.103,0.000,0.000,0.000,1.000,8.000
Parch,891.000,0.382,0.806,0.000,0.000,0.000,0.000,6.000
Fare,891.000,32.204,49.693,0.000,7.910,14.454,31.000,512.329
Age_standard_scaler,714.000,0.000,1.001,-2.017,-0.660,-0.117,0.572,3.465
Age_robuts_scaler,714.000,0.095,0.813,-1.543,-0.441,0.000,0.559,2.909
Age_min_max_scaler,714.000,0.368,0.183,0.000,0.248,0.347,0.472,1.000


* Scale yaptığımızda verinin dağılımı değişmez sadece ölçeği değişir.